In [ ]:
%pip install music21 matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from music21 import converter


# Load your MIDI file
midi_path = 'archive\.38 Special\Caught Up In You.mid'
midi_data = converter.parse(midi_path)



In [ ]:
from music21 import converter, note, chord

def get_notes_and_durations(file_path):
    midi_data = converter.parse(file_path)
    elements = midi_data.recurse().notesAndRests  # Get notes and chords

    note_data = []
    for element in elements:
        if isinstance(element, note.Note):
            note_data.append((element.pitch.midi, element.offset, element.duration.quarterLength))
        elif isinstance(element, chord.Chord):
            for pitch in element.pitches:
                note_data.append((pitch.midi, element.offset, element.duration.quarterLength))

    return note_data

In [ ]:
note_data = get_notes_and_durations(midi_path)

import matplotlib.pyplot as plt
import numpy as np

def plot_piano_roll(note_data, start_time=0, end_time=None, min_pitch=21, max_pitch=108):
    fig, ax = plt.subplots(figsize=(15, 8))

    # Filtering notes within the specified time and pitch range
    note_data = [note for note in note_data if start_time <= note[1] <= end_time] if end_time else note_data
    note_data = [note for note in note_data if min_pitch <= note[0] <= max_pitch]

    for note in note_data:
        pitch, start, duration = note
        ax.barh(pitch, duration, left=start, height=1, edgecolor='black')

    ax.set_yticks(np.arange(min_pitch, max_pitch + 1, 1))
    ax.set_yticklabels([str(p) for p in range(min_pitch, max_pitch + 1, 1)])
    ax.invert_yaxis()  # Invert y-axis to have low pitches at the bottom
    ax.set_xlabel('Time (Quarter Length)')
    ax.set_ylabel('Pitch (MIDI Number)')
    ax.set_title('Piano Roll')

    plt.show()

plot_piano_roll(note_data, start_time=0, end_time=100)  # Adjust 'end_time' as needed


In [ ]:
instruments = midi_data.recurse().getElementsByClass('Instrument')

for i, instrument in enumerate(instruments):
    print(f"Track {i}: {instrument.instrumentName}")

In [ ]:
chords = midi_data.chordify()
chord_names = [c.pitchedCommonName for c in chords.recurse().getElementsByClass('Chord')]
chord_names

In [ ]:
tempos = midi_data.metronomeMarkBoundaries()
tempo_times = [tempo[0] for tempo in tempos]
tempo_values = [tempo[2].number for tempo in tempos]

plt.figure(figsize=(10, 4))
plt.plot(tempo_times, tempo_values, label='Tempo', color='blue')
plt.xlabel('Time (s)')
plt.ylabel('Tempo (BPM)')
plt.title('Tempo Changes Over Time')
plt.legend()
plt.show()


In [ ]:
time_signatures = midi_data.recurse().getElementsByClass('TimeSignature')
for ts in time_signatures:
    print(ts.ratioString)
